In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
from urllib.parse import urlparse, urljoin

In [3]:
# collect the Acreage data
import requests

for year in range(2009, 2019):
    if not os.path.isfile(f"data/size_data/{year}.xlsx"):
        dls = f"https://www.nps.gov/subjects/lwcf/upload/NPS-Acreage-12-31-{year}.xlsx"
        resp = requests.get(dls)
        
        output = open(f"data/size_data/{year}.xlsx", 'wb+')
        output.write(resp.content)
        output.close()

In [1]:
# get list of all park names that are in both files
from helpers import get_all_parks

all_park_names = get_all_parks()

data/visit_data\2009.csv
data/visit_data\2010.csv
data/visit_data\2011.csv
data/visit_data\2012.csv
data/visit_data\2013.csv
data/visit_data\2014.csv
data/visit_data\2015.csv
data/visit_data\2016.csv
data/visit_data\2017.csv
data/visit_data\2018.csv


In [4]:
def get_lat_long(name, lat_long_df):
    api_key = os.environ['GOOGLE_API_KEY']
    api_response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'.format(name, api_key))
    api_response_dict = api_response.json()
    
    if api_response_dict['status'] == 'OK':
        latitude = api_response_dict['results'][0]['geometry']['location']['lat']
        longitude = api_response_dict['results'][0]['geometry']['location']['lng']
        #print('Latitude:', latitude)
        #print('Longitude:', longitude)
        lat_long_df = lat_long_df.append({'ParkName': name, 'latitude': latitude, 'longitude': longitude}, ignore_index=True)
        return lat_long_df
        

In [6]:
if os.path.isfile("./data/lat_lon_data.csv"):
    merged = pd.read_csv("./data/lat_lon_data.csv")
else:
    lat_long_df = pd.DataFrame(columns=['ParkName', 'latitude', 'longitude'])
    
    for park in all_park_names:
        print(park)
        lat_long_df = get_lat_long(park, lat_long_df)
        
    lat_long_df.to_csv("./data/lat_lon_data.csv")

KOBUK VALLEY NP
BRYCE CANYON NP
LASSEN VOLCANIC NP
GLEN CANYON NRA
STEAMTOWN NHS
BLUE RIDGE PKWY
GRAND TETON NP
SAGUARO NP
REDWOOD NP
VANDERBILT MANSION NHS
AGATE FOSSIL BEDS NM
CASTLE CLINTON NM
MONTEZUMA CASTLE NM
ROSS LAKE NRA
PICTURED ROCKS NL
MANASSAS NBP
PISCATAWAY PARK
NATURAL BRIDGES NM
PETRIFIED FOREST NP
KEWEENAW NHP
CAPE KRUSENSTERN NM
ACADIA NP
VICKSBURG NMP
BLUESTONE NSR
GAULEY RIVER NRA
GETTYSBURG NMP
KALAUPAPA NHP
GOVERNORS ISLAND NM
MAMMOTH CAVE NP
CANE RIVER CREOLE NHP
MANZANAR NHS
GRAND PORTAGE NM
WOMEN'S RIGHTS NHP
GRANT-KOHRS RANCH NHS
JEWEL CAVE NM
RIVER RAISIN NBP
PADRE ISLAND NS
DE SOTO NMEM
CUMBERLAND ISLAND NS
SPRINGFIELD ARMORY NHS
BANDELIER NM
COLONIAL NHP
NAVAJO NM
MINUTEMAN MISSILE NHS
GREAT BASIN NP
RUSSELL CAVE NM
LINCOLN MEMORIAL
NORTH CASCADES NP
AMISTAD NRA
ARKANSAS POST NMEM
ROCK CREEK PARK
CURECANTI NRA
HAWAII VOLCANOES NP
MANHATTAN PROJECT NHP
FORD'S THEATRE NHS
MOORES CREEK NB
TONTO NM
HOT SPRINGS NP
WAR IN THE PACIFIC NHP
DINOSAUR NM
JIMMY CARTER 

In [10]:
for year in range(2009, 2019):
    visiting_data = pd.read_csv(f"data/visit_data/{year}.csv", header=2)
    visiting_data["ParkName"] = visiting_data["ParkName"].apply(lambda x: x.upper())
    print(visiting_data.head())
    
    acreage_data = pd.read_excel(f"data/size_data/{year}.xlsx", header=1)
    acreage_data.rename(columns={"Area Name": "ParkName"}, inplace=True)
    acreage_data["ParkName"] = acreage_data["ParkName"].apply(lambda x : x.upper())
    print(acreage_data.head())
    
    acreage_data.set_index("ParkName", inplace=True)
    visiting_data.set_index("ParkName", inplace=True)
    
    merged = visiting_data.join(acreage_data)
    merged = merged.dropna()
    
    lat_long_df = pd.read_csv(f"data/lat_lon_data.csv")
    lat_long_df.set_index("ParkName", inplace=True)
    
    merged = merged.join(lat_long_df)
    merged = merged.dropna()

    print(merged.head())
    print(merged.shape)
    
    merged.reset_index(inplace = True) 
    merged.to_csv(f"./data/merged_data/merged_data_{year}.csv")
    

                   ParkName  Rank       Value PercentOfTotal
0           BLUE RIDGE PKWY     1  15,936,316          5.58%
1           GOLDEN GATE NRA     2  15,036,372          5.27%
2  GREAT SMOKY MOUNTAINS NP     3   9,491,437          3.32%
3               GATEWAY NRA     4   9,010,522          3.16%
4             LAKE MEAD NRA     5   7,668,689          2.69%
                   ParkName State Region  NPS Fee Acres  \
0     A LINCOLN BIRTHPL NHP    KY     SE         344.50   
1                 ACADIA NP    ME     NE       35694.17   
2                 ADAMS NHP    MA     NE           9.17   
3  AFRICAN BURIAL GROUND NM    NY     NE           0.35   
4      AGATE FOSSIL BEDS NM    NE     MW        2270.33   

   NPS Less Than Fee Acres  Other Federal Fee Acres  Subtotal Federal Acres  \
0                     0.00                      0.0                  344.50   
1                 10624.09                     21.8                46340.06   
2                     0.00                